In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import folium

In [2]:
df_zomato = pd.read_csv('zomato.csv',encoding = "ISO-8859-1")

df_zomato.head()

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,Average Cost for two,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",1100,Botswana Pula(P),Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,1200,Botswana Pula(P),Yes,No,No,No,3,4.5,Dark Green,Excellent,591
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",4000,Botswana Pula(P),Yes,No,No,No,4,4.4,Green,Very Good,270
3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,"Japanese, Sushi",1500,Botswana Pula(P),No,No,No,No,4,4.9,Dark Green,Excellent,365
4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,"Japanese, Korean",1500,Botswana Pula(P),Yes,No,No,No,4,4.8,Dark Green,Excellent,229


In [3]:
df_zomato.dropna(inplace=True)

In [4]:
df_zomato_new_delhi = df_zomato[df_zomato['City'] == 'New Delhi']

In [5]:
required_columns = ['Locality']

df_filtered = df_zomato_new_delhi[required_columns]

df_filtered.head()

,Locality
2560,Aaya Nagar
2561,Adchini
2562,Adchini
2563,Adchini
2564,Adchini


In [8]:
df_filtered.drop_duplicates(subset=None, keep='first', inplace=True)

df_filtered.shape

(254, 1)

In [9]:
df_filtered = df_filtered.reset_index()

In [10]:
df_filtered.head()

,index,Locality
0,2560,Aaya Nagar
1,2561,Adchini
2,2574,"Aditya Mega Mall, Karkardooma"
3,2578,Aerocity
4,2582,"Aggarwal City Mall, Pitampura"


In [11]:
df_filtered.drop(columns=['index'], axis=1, inplace=True)
df_filtered.head()

,Locality
0,Aaya Nagar
1,Adchini
2,"Aditya Mega Mall, Karkardooma"
3,Aerocity
4,"Aggarwal City Mall, Pitampura"


In [12]:
df_filtered.shape

(254, 1)

In [13]:
new_columns = ['Locality', 'Latitude', 'Longitude']

df_localities = pd.DataFrame(columns=new_columns)

df_localities.head()

,Locality,Latitude,Longitude


In [14]:
for index, row in df_filtered.iterrows():
    latitude = None
    longitude = None
    lat_lng_coords = None

    locator = Nominatim(user_agent='myGeocoder')
    location = locator.geocode('{}, New Delhi'.format(row['Locality']))

    if location is not None:
        latitude = location.latitude
        longitude = location.longitude
    else:
        # if no coordinates are found, then skip this locality
        print('No coordinates found for {}, New Delhi'.format(row['Locality']))
        continue

    df_localities = df_localities.append({'Locality': row['Locality'],
                                        'Longitude': longitude,
                                        'Latitude': latitude}, ignore_index=True)


df_localities.head()

No coordinates found for Aaya Nagar, New Delhi
No coordinates found for Aditya Mega Mall, Karkardooma, New Delhi
No coordinates found for Aggarwal City Mall, Pitampura, New Delhi
No coordinates found for Aggarwal City Plaza, Rohini, New Delhi
No coordinates found for Andaz Delhi, Aerocity, New Delhi
No coordinates found for Ansal Plaza Mall, Khel Gaon Marg, New Delhi
No coordinates found for ARSS Mall, Paschim Vihar, New Delhi
No coordinates found for Ashok Vihar Phase 1, New Delhi
No coordinates found for Ashok Vihar Phase 2, New Delhi
No coordinates found for Ashok Vihar Phase 3, New Delhi
No coordinates found for Basant Lok Market, Vasant Vihar, New Delhi
No coordinates found for Bellagio, Ashok Vihar Phase 2, New Delhi
No coordinates found for Best Western Taurus Hotel, Mahipalpur, New Delhi
No coordinates found for Centaur Hotel, Aerocity, New Delhi
No coordinates found for City Centre Mall, Rohini, New Delhi
No coordinates found for City Square Mall, Rajouri Garden, New Delhi
No 

,Locality,Latitude,Longitude
0,Adchini,28.537024,77.198227
1,Aerocity,28.547990,77.121240
2,Alaknanda,28.529336,77.251632
3,"Ambience Mall, Vasant Kunj",28.541245,77.155108
4,Anand Lok,28.557292,77.219122


In [15]:
df_localities.shape

(151, 3)

In [16]:
address = 'New Delhi'

geolocator = Nominatim(user_agent="nd_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New Delhi is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New Delhi is 28.6138954, 77.2090057.


In [17]:
map_newdelhi = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for locality, lat, lng in zip(df_localities['Locality'], df_localities['Latitude'], df_localities['Longitude']):
    label = '{}, New Delhi'.format(locality)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newdelhi)  

map_newdelhi

In [18]:
df_localities.loc[0, 'Locality']

'Adchini'

In [19]:
locality_latitude = df_localities.loc[0, 'Latitude'] # neighborhood latitude value
locality_longitude = df_localities.loc[0, 'Longitude'] # neighborhood longitude value

locality_name = df_localities.loc[0, 'Locality'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(locality_name, 
                                                               locality_latitude, 
                                                               locality_longitude))

Latitude and longitude values of Adchini are 28.537024000000002, 77.19822731852571.


In [20]:
CLIENT_ID = 'I3QEN1ICKGB4W1JQFCBQN42PO4QRL5DY0OMXW1VSSXF4IRG4' # your Foursquare ID
CLIENT_SECRET = 'SYZKRVWYN4MUMTKOZKPWTTCV5P0WQHKJS1OW0EIJCLSTYYWD' # your Foursquare Secret
VERSION = '20200630' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: I3QEN1ICKGB4W1JQFCBQN42PO4QRL5DY0OMXW1VSSXF4IRG4
CLIENT_SECRET:SYZKRVWYN4MUMTKOZKPWTTCV5P0WQHKJS1OW0EIJCLSTYYWD


In [21]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    locality_latitude, 
    locality_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=I3QEN1ICKGB4W1JQFCBQN42PO4QRL5DY0OMXW1VSSXF4IRG4&client_secret=SYZKRVWYN4MUMTKOZKPWTTCV5P0WQHKJS1OW0EIJCLSTYYWD&v=20200630&ll=28.537024000000002,77.19822731852571&radius=500&limit=100'

In [22]:
results = requests.get(url).json()

In [23]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [24]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,100% Rock,Pub,28.535552,77.197523
1,Cafe Coffee Day,Café,28.538589,77.198683
2,Kuzart Lane,Café,28.537990,77.198368
3,Waves Restaurant,Indian Restaurant,28.538582,77.198771
4,Southy,Indian Restaurant,28.537476,77.198048


In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Locality', 
                  'Locality Latitude', 
                  'Locality Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [26]:
newdelhi_venues = getNearbyVenues(names=df_localities['Locality'],
                                   latitudes=df_localities['Latitude'],
                                   longitudes=df_localities['Longitude']
                                  )

Adchini
Aerocity
Alaknanda
Ambience Mall, Vasant Kunj
Anand Lok
Anand Vihar
Asaf Ali Road
Barakhamba Road
Bhikaji Cama Place
Chanakyapuri
Chander Nagar
Chandni Chowk
Chawri Bazar
Chhatarpur
Chittaranjan Park
Civil Lines
Connaught Place
Daryaganj
Defence Colony
Delhi Cantt.
Dilshad Garden
DLF Emporio Mall, Vasant Kunj
DLF Promenade Mall, Vasant Kunj
Dr. Zakir Hussain Marg
Durga Puri
East of Kailash
East Patel Nagar
Friends Colony
Garden of Five Senses, Saket
Geeta Colony
Green Park
GTB Nagar
Hauz Khas
Hauz Khas Village
Holiday Inn, Aerocity
INA
India Gate
IP Extension
ITO
Jail Road
Jama Masjid
Janakpuri
Jangpura
Janpath
Jasola
Jaypee Vasant Continental, Vasant Vihar
JNU
Jor Bagh
Kailash Colony
Kalkaji
Kamla Nagar
Kapashera
Karampura
Karkardooma
Karol Bagh
Kashmiri Gate
Khan Market
Kirti Nagar
Krishna Nagar
Lado Sarai
Lajpat Nagar 1
Lajpat Nagar 2
Lawrence Road
Laxmi Nagar
Le Meridien, Janpath
Living Style Mall, Jasola
Lodhi Colony
Lodhi Road
Mahipalpur
Maidens Hotel, Civil Lines
Majnu k

In [27]:
print(newdelhi_venues.shape)
newdelhi_venues.head()

(1459, 7)


,Locality,Locality Latitude,Locality Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Adchini,28.537024,77.198227,100% Rock,28.535552,77.197523,Pub
1,Adchini,28.537024,77.198227,Cafe Coffee Day,28.538589,77.198683,Café
2,Adchini,28.537024,77.198227,Kuzart Lane,28.537990,77.198368,Café
3,Adchini,28.537024,77.198227,Waves Restaurant,28.538582,77.198771,Indian Restaurant
4,Adchini,28.537024,77.198227,Southy,28.537476,77.198048,Indian Restaurant


In [29]:
newdelhi_venues.groupby('Locality').count()

,Locality Latitude,Locality Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Locality,,,,,,
Adchini,5,5,5,5,5,5
Aerocity,17,17,17,17,17,17
Alaknanda,11,11,11,11,11,11
"Ambience Mall, Vasant Kunj",46,46,46,46,46,46
Anand Lok,9,9,9,9,9,9
Anand Vihar,7,7,7,7,7,7
Asaf Ali Road,2,2,2,2,2,2
Barakhamba Road,16,16,16,16,16,16
Bhikaji Cama Place,12,12,12,12,12,12


In [31]:
print('There are {} uniques categories.'.format(len(newdelhi_venues['Venue Category'].unique())))

There are 183 uniques categories.
